In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Earthquake-priv/Data.xlsx


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('seaborn-deep')
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['figure.figsize'] = (12, 8)

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 400)
import warnings
warnings.filterwarnings('ignore')
import sklearn.base as skb
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import sklearn.utils as sku
import sklearn.linear_model as sklm
import sklearn.neighbors as skn
import sklearn.ensemble as ske
import catboost as cb
import scipy.stats as sstats
import random
seed = 12
np.random.seed(seed)

from datetime import date

import pandas_profiling as pp

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


print(tf.__version__)

# Make NumPy printouts easier to read.


2.6.4


In [4]:
# !pip install tensorflow-addons

In [5]:
dataset = pd.read_excel("/kaggle/input/Earthquake-priv/Data.xlsx")

In [6]:
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,0.04,0.03,0.02,0.46,0.02,0.15
1,0.04,0.03,0.02,0.52,0.02,0.24
2,0.13,0.07,0.10,0.15,0.02,3.33
3,0.10,0.05,0.07,0.21,0.02,1.88
4,0.06,0.04,0.04,0.32,0.02,0.90


In [7]:
# profile = pp.ProfileReport(dataset, title='Pandas Profiling Report', explorative=True)
# profile.to_file("profile.html")

In [8]:
dataset.isna().sum()

PGA       0
PGV       0
Ia        0
CAR       0
Ky        0
P_Disp    0
dtype: int64

In [9]:
dataset.describe()

,PGA,PGV,Ia,CAR,Ky,P_Disp
count,13707.00,13707.00,13707.00,13707.00,13707.00,13707.00
mean,0.22,0.22,0.88,0.45,0.08,14.32
std,0.21,0.24,1.74,0.24,0.08,51.74
min,0.02,0.00,0.00,0.01,0.02,0.01
25%,0.08,0.07,0.10,0.24,0.02,0.19
50%,0.15,0.13,0.28,0.43,0.05,1.20
75%,0.29,0.28,0.87,0.64,0.10,7.34
max,1.78,2.63,22.75,0.97,0.40,1883.85


In [10]:
dataset = dataset.drop(dataset[(dataset.P_Disp > 150) | (dataset.P_Disp < 0.015)].index)
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,0.04,0.03,0.02,0.46,0.02,0.15
1,0.04,0.03,0.02,0.52,0.02,0.24
2,0.13,0.07,0.10,0.15,0.02,3.33
3,0.10,0.05,0.07,0.21,0.02,1.88
4,0.06,0.04,0.04,0.32,0.02,0.90


In [11]:
sns.pairplot(dataset[['PGA', 'PGV', 'Ia', 'CAR','Ky','P_Disp' ]], diag_kind='kde')

In [12]:
dataset = np.log(dataset)
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,-3.13,-3.66,-4.17,-0.78,-3.91,-1.87
1,-3.27,-3.45,-3.88,-0.65,-3.91,-1.41
2,-2.01,-2.63,-2.29,-1.90,-3.91,1.20
3,-2.34,-2.91,-2.59,-1.57,-3.91,0.63
4,-2.78,-3.31,-3.27,-1.13,-3.91,-0.11


In [13]:
# profile = pp.ProfileReport(dataset, title='Pandas Profiling Report', explorative=True)
# profile.to_file("profile.html")

In [14]:
# np.exp(-3.13)

In [15]:
# sns.histplot(dataset.PGA)

In [16]:
# sns.histplot(dataset.PGV)

In [17]:
# sns.histplot(dataset.Ia)

In [18]:
# sns.histplot(dataset.CAR)

In [19]:
# sns.histplot(dataset.Ky)

In [20]:
# sns.histplot(dataset.P_Disp)


In [21]:
sns.pairplot(dataset[['PGA', 'PGV', 'Ia', 'CAR','Ky','P_Disp' ]], diag_kind='kde')

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [23]:
dataset=MultiColumnLabelEncoder(columns = ['Ky']).fit_transform(dataset)
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,-3.13,-3.66,-4.17,-0.78,0,-1.87
1,-3.27,-3.45,-3.88,-0.65,0,-1.41
2,-2.01,-2.63,-2.29,-1.90,0,1.20
3,-2.34,-2.91,-2.59,-1.57,0,0.63
4,-2.78,-3.31,-3.27,-1.13,0,-0.11


In [24]:
dataset.describe()

,PGA,PGV,Ia,CAR,Ky,P_Disp
count,13129.00,13129.00,13129.00,13129.00,13129.00,13129.00
mean,-1.88,-2.00,-1.27,-1.00,1.95,0.20
std,0.86,0.95,1.51,0.68,2.17,2.24
min,-3.81,-5.53,-5.77,-4.24,0.00,-4.20
25%,-2.47,-2.66,-2.31,-1.39,0.00,-1.54
50%,-1.89,-2.01,-1.29,-0.84,1.00,0.20
75%,-1.26,-1.29,-0.16,-0.46,3.00,1.93
max,0.58,0.97,3.12,-0.04,9.00,5.01


In [31]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)


In [32]:
train_features = X_train
test_features = X_test

train_labels = y_train
test_labels = y_test

In [33]:
train_features.head()

,PGA,PGV,Ia,CAR,Ky
3148,-2.76,-2.15,-2.74,-1.15,0
8641,-2.31,-2.57,-2.05,-0.28,2
12574,-0.53,-0.79,1.36,-0.86,6
10801,-2.13,-2.94,-1.98,-0.18,3
11352,-1.10,-1.99,-0.62,-0.80,4


In [24]:
# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(np.array(train_features))
# print(normalizer.mean.numpy())

In [25]:
# input_data  = np.array([[4 , 5 , 6 , 7, 8]])
# normalizer(input_data)

In [25]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.plot(history.history['Std_err'], label='Std_err')
  plt.ylim([0, 5])
  plt.xlabel('Epoch')
  plt.ylabel('Error [P_Disp]')
  plt.legend()
  plt.grid(True)

In [26]:
from keras import backend as k
# def matthews_correlation(y_true, y_pred):
#     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
#     y_pred_neg = 1 - y_pred_pos

#     y_pos = K.round(K.clip(y_true, 0, 1))
#     y_neg = 1 - y_pos

#     tp = K.sum(y_pos * y_pred_pos)
#     tn = K.sum(y_neg * y_pred_neg)

#     fp = K.sum(y_neg * y_pred_pos)
#     fn = K.sum(y_pos * y_pred_neg)

#     numerator = (tp * tn - fp * fn)
#     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

#     return numerator / (denominator + K.epsilon())

def Std_err(y_true, y_pred):
    err  = y_pred - y_true
    mean_err = k.mean(err)
    d = err - mean_err
    ans = k.sqrt(k.mean(k.sum(k.square(d))))
    return ans


                
    


In [27]:
def Corr_R(y_true, y_pred):
    y_true_mean = k.mean(y_true)
    y_pred_mean = k.mean(y_pred)
    num = k.sum((y_true-y_true_mean)*(y_pred-y_pred_mean))
    den = k.sqrt((k.sum(k.square(y_true-y_true_mean)))*(k.sum(k.square(y_pred-y_pred_mean))))
    return num/den

In [28]:
def build_and_compile_model(n):
  model = keras.Sequential([
#       norm,
      layers.Dense(8, input_shape=(n,)),
      layers.Dense(8, activation='relu'),
#       layers.Dense(8, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001), metrics = ['MeanAbsoluteError',Std_err,Corr_R])
  return model

In [29]:
dnn_model = build_and_compile_model(5)
dnn_model.summary()

2022-10-13 07:10:21.224527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:21.333486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:21.334770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:21.337003: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 48        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


2022-10-13 07:10:21.337383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:21.338469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:21.339476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:23.671073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 07:10:23.671926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [34]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    verbose=1, epochs=50)

2022-10-13 07:11:11.702931: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
259/259 [==============================] - 2s 3ms/step - loss: 1.5153 - mean_absolute_error: 1.5153 - Std_err: 10.1411 - Corr_R: 0.5503 - val_loss: 0.7187 - val_mean_absolute_error: 0.7187 - val_Std_err: 4.9578 - val_Corr_R: 0.9197
Epoch 2/50
259/259 [==============================] - 1s 3ms/step - loss: 0.6467 - mean_absolute_error: 0.6467 - Std_err: 4.6121 - Corr_R: 0.9292 - val_loss: 0.5926 - val_mean_absolute_error: 0.5926 - val_Std_err: 4.1758 - val_Corr_R: 0.9422
Epoch 3/50
259/259 [==============================] - 1s 3ms/step - loss: 0.6017 - mean_absolute_error: 0.6017 - Std_err: 4.4014 - Corr_R: 0.9344 - val_loss: 0.5670 - val_mean_absolute_error: 0.5670 - val_Std_err: 4.0492 - val_Corr_R: 0.9457
Epoch 4/50
259/259 [==============================] - 1s 3ms/step - loss: 0.5704 - mean_absolute_error: 0.5704 - Std_err: 4.1812 - Corr_R: 0.9416 - val_loss: 0.5405 - val_mean_absolute_error: 0.5405 - val_Std_err: 3.7211 - val_Corr_R: 0.9551
Epoch 5/50
259/259 [===========

In [32]:
plot_loss(history)

In [33]:
# test_results =[]
# test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

In [34]:
input_data = np.log(np.array([[0.0621619,0.036424837,0.038161434,0.321740487,0.02]]))
print(input_data)
# input  = normalizer(input_data)
# print(input)
t = np.exp(dnn_model.predict(input_data))
t

[[-2.77801301 -3.3125044  -3.26592985 -1.13401    -3.91202301]]


array([[0.57475644]], dtype=float32)

PGA, PGV, Ia

----------------------------------------------------------------------------------------------------------------------------------------________________________________________________________________________________________________________________________________________

In [35]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','Ia']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [36]:
X.head()

,PGA,PGV,Ia
0,-3.13,-3.66,-4.17
1,-3.27,-3.45,-3.88
2,-2.01,-2.63,-2.29
3,-2.34,-2.91,-2.59
4,-2.78,-3.31,-3.27


In [37]:
dnn_model = build_and_compile_model(3)
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [38]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

Epoch 1/25
263/263 [==============================] - 1s 4ms/step - loss: 1.7618 - mean_absolute_error: 1.7618 - Std_err: 11.6188 - Corr_R: 0.2878 - val_loss: 1.5701 - val_mean_absolute_error: 1.5701 - val_Std_err: 10.5466 - val_Corr_R: 0.5351
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 1.5719 - mean_absolute_error: 1.5719 - Std_err: 10.6421 - Corr_R: 0.5146 - val_loss: 1.5484 - val_mean_absolute_error: 1.5484 - val_Std_err: 10.3267 - val_Corr_R: 0.5626
Epoch 3/25
263/263 [==============================] - 1s 3ms/step - loss: 1.5498 - mean_absolute_error: 1.5498 - Std_err: 10.5048 - Corr_R: 0.5329 - val_loss: 1.5269 - val_mean_absolute_error: 1.5269 - val_Std_err: 10.2266 - val_Corr_R: 0.5745
Epoch 4/25
263/263 [==============================] - 1s 3ms/step - loss: 1.5373 - mean_absolute_error: 1.5373 - Std_err: 10.4431 - Corr_R: 0.5427 - val_loss: 1.5238 - val_mean_absolute_error: 1.5238 - val_Std_err: 10.1528 - val_Corr_R: 0.5814
Epoch 5/25
263/263 [====

In [39]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','CAR']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

dnn_model = build_and_compile_model(3)
dnn_model.summary()


history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

plot_loss(history)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
263/263 [==============================] - 1s 4ms/step - loss: 1.2868 - mean_absolute_error: 1.2868 - Std_err: 8.5924 - Corr_R: 0.7836 - val_loss: 0.7619 - val_mean_absolute_error: 0.7619 - val_Std_err: 5.2249 - val_Corr_R: 0.9156
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.6550 - mean_absolute_error: 0.6550 - Std_err: 4.6888 - Corr_R: 0.926

In [40]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','Ky']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

dnn_model = build_and_compile_model(3)
dnn_model.summary()


history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

plot_loss(history)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
263/263 [==============================] - 1s 3ms/step - loss: 2.1897 - mean_absolute_error: 2.1897 - Std_err: 12.7629 - Corr_R: -0.0256 - val_loss: 1.7984 - val_mean_absolute_error: 1.7984 - val_Std_err: 11.7478 - val_Corr_R: 0.6486
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 1.7111 - mean_absolute_error: 1.7111 - Std_err: 11.1098 - Corr_R: 0

In [41]:
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,-3.13,-3.66,-4.17,-0.78,-3.91,-1.87
1,-3.27,-3.45,-3.88,-0.65,-3.91,-1.41
2,-2.01,-2.63,-2.29,-1.90,-3.91,1.20
3,-2.34,-2.91,-2.59,-1.57,-3.91,0.63
4,-2.78,-3.31,-3.27,-1.13,-3.91,-0.11


In [42]:
X_train.head()

,PGA,PGV,Ky
11124,-1.38,-0.72,-1.90
9323,-2.02,-1.62,-2.59
7337,-2.31,-2.82,-3.00
1802,-2.25,-2.51,-3.91
7883,-2.35,-3.10,-2.59


In [43]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','Ia','CAR']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

dnn_model = build_and_compile_model(4)
dnn_model.summary()


history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

plot_loss(history)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 9         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
263/263 [==============================] - 1s 3ms/step - loss: 1.4016 - mean_absolute_error: 1.4016 - Std_err: 9.2813 - Corr_R: 0.7563 - val_loss: 0.8144 - val_mean_absolute_error: 0.8144 - val_Std_err: 5.7053 - val_Corr_R: 0.8912
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.7151 - mean_absolute_error: 0.7151 - Std_err: 5.0719 - Corr_R: 0.912

In [44]:
plot_loss(history)